In [2]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
import time


In [17]:
window = tk.Tk()
window.title("Hand Written Digit Recognition")

l1 = tk.Label(window,text="Digit",font=('Algerian',20))
l1.place(x=5,y=0)

t1 = tk.Entry(window,width=20,border = 5)
t1.place(x=150,y=0)
 
#count = len([name for name in os.listdir('.') if os.path.isfile(name)])   
#len(os.listdir(directory))

click = False
def screen_capture():
    global click
    import pyscreenshot as ImageGrab
    import os
    n = int(input("Enter the number of n: "))
    global count
    os.startfile ("C:/ProgramData/Microsoft/Windows/Start Menu/Programs/Accessories/Paint")
    s1 = t1.get()
    os.chdir("C:/Users/brajs/Digit_Recognizer/captured_images/")
    
    #if click == False:
        #os.mkdir(s1)
    
    if os.path.isdir(s1) == False:
        os.mkdir(s1)
    os.chdir("C:/Users/brajs/Digit_Recognizer/")
    click = True
    images_folder = "captured_images/"+s1+"/"
    time.sleep(15)
    for i in range(n):
        time.sleep(8)
        im = ImageGrab.grab(bbox = (60,170,400,550))
        print("Saved...........",len(os.listdir(images_folder)))
        im.save(images_folder + str(len(os.listdir(images_folder)))+'.png')
        print("Clear Screen now and re-draw now........")
  
    messagebox.showinfo("Result","Capturing screen is completed !!!!")

def generate_dataset():
    import cv2 
    import csv
    import glob
    header = ["label"]
    for i in range(0,784):
        header.append("pixel"+str(i))
    with open('dataset2.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(header)
    for label in range(10):
        dirList = glob.glob("captured_images/"+str(label)+"/*.png")
        for img_path in dirList :
            im = cv2.imread(img_path)
            im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
            im_gray = cv2.GaussianBlur(im_gray,(15,15),0)
            roi = cv2.resize(im_gray, (28,28),interpolation = cv2.INTER_AREA)
            data = []
            data.append(label)
            rows, cols = roi.shape
            for i in range(rows):
                for j in range(cols):
                    k = roi[i,j]/255
                    
                    #if k>100:
                        #k = 1 
                    #else:
                        #k = 0
                    
                    data.append(k)
            with open('dataset2.csv','a') as f:
                writer = csv.writer(f)
                writer.writerow(data)
    
    messagebox.showinfo("Result","generating dataset")
    
def train_save_accuracy():
    
    import pandas as pd
    from sklearn.utils import shuffle
    
    data = pd.read_csv('dataset2.csv')
    data = shuffle(data)
    x = data.drop(["label"], axis = 1)
    y = data["label"]
    
    from sklearn.model_selection import train_test_split
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2)
    
    import joblib
    from sklearn.svm import SVC
    
    classifier = SVC(kernel = "linear",random_state = 6)
    classifier.fit(x_train,y_train)
    joblib.dump(classifier, "model/Digit_Recognizer")
    
    from sklearn import metrics
    
    prediction = classifier.predict(x_test)
    acc = metrics.accuracy_score(prediction, y_test)
    
    messagebox.showinfo("Result", f"Your accuracy is:{acc}")
    
def prediction():
    import pyscreenshot as ImageGrab
    import os
    import joblib
    import cv2
    from keras.models import load_model
    import numpy as np

    os.startfile ("C:/ProgramData/Microsoft/Windows/Start Menu/Programs/Accessories/Paint")
    
    model = joblib.load("model/Digit_Recognizer")
    image_folder = "img/"
    time.sleep(15)
    while True:
        img = ImageGrab.grab(bbox = (60,170,400,550))
        img.save(image_folder+"img.png")
        im = cv2.imread(image_folder+"img.png")
        im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im_gray = cv2.GaussianBlur(im_gray,(15,15),0)
        ret, im_th = cv2.threshold(im_gray, 100,255,cv2.THRESH_BINARY)
        roi = cv2.resize(im_th, (28,28), interpolation = cv2.INTER_AREA)
        rows, cols = roi.shape
        x = []
        for i in range(rows):
            for j in range(cols):
                k = roi[i,j]/255
                
                #if k>100:
                   # k = 1 
                #else:
                    #k = 0
                x.append(k)
        
                    
        predictions = model.predict([x])
        print("prediction: ", predictions[0])
        cv2.putText(im, "prediction is: " + str(predictions[0]), (20,20), 0, 0.8, (0,255,0),2, cv2.LINE_AA)
        cv2.startWindowThread()
        cv2.namedWindow("Result")
        cv2.imshow("Result", im)
        cv2.waitKey(10000)
        if cv2.waitKey(1) == 27:    # esc button
            break
        
        
    cv2.destroyAllWindows()
    
               
    
b1 = tk.Button(window,text = "1. Open Paint and capture the screen", font = ('Algerian',15),bg = "orange",fg = "black",command = screen_capture)
b1.place(x = 5, y = 50)

b2 = tk.Button(window,text = "2. To Generate Dataset", font = ('Algerian',15),bg = "pink",fg = "blue",command = generate_dataset)
b2.place(x = 5, y = 100)

b3 = tk.Button(window,text = "3. Train the Model, save it and calculate Accuracy", font = ('Algerian',15),bg = "green",fg = "white",command = train_save_accuracy)
b3.place(x = 5, y = 150)

b4 = tk.Button(window,text = "4. Live Prediction", font = ('Algerian',15),bg = "white",fg = "red",command = prediction)
b4.place(x = 5, y = 200)

window.geometry("600x300")
window.mainloop()

prediction:  0
prediction:  9
prediction:  4
prediction:  3
prediction:  9
prediction:  9
prediction:  9
